In [8]:
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

In [2]:
class Loader:

    def __init__(self, files):
        self.files = files

    def get_data(self):
        dfs = []
        for file in self.files:
            df = self.load_past_matches(file)
            dfs.append(df)
        
        df_join = self.join_data(dfs[0], dfs[1])

        return df_join

    def load_past_matches(self, file):
        df = pd.read_csv(f'../data/{file}')
        df.drop('Unnamed: 0', axis=1, inplace=True)
        df['date'] = pd.to_datetime(df['date']).dt.date

        return df

    def join_data(self, df1, df2):
        df = pd.merge(df1, df2,  how='inner',
            left_on=['league', 'date','team', 'opponent', 'home'],
            right_on=['league', 'date','team', 'opponent', 'home'])
        df.sort_values(by=['date', 'league', 'team', 'opponent'], inplace=True)
              
        return df

In [3]:
FILES = ["elos_matches.csv", "goals_matches.csv"]
loader = Loader(FILES)
data = loader.get_data()

In [18]:
def filter_to_season_and_league(df, season, league):
    start_date = pd.to_datetime(f"{season}-07-30").date()
    df = df[df['date'] >= start_date]
    df = df[df['league'] == league]
    df.reset_index(inplace=True, drop=True)
    return df

In [22]:
data = filter_to_season_and_league(data, '2022', 'Serie C, Girone B')
data.drop([
    'team_goals_scored',
    'team_goals_conceded',
    'opponent_goals_scored',
    'opponent_goals_conceded'
], axis=1, inplace=True)
data.head()

,league,date,team,opponent,result,elo_team,elo_opponent,elo_diff,home,team_goals_scored_avg,...,league_home_goals_conceded,league_away_goals_conceded,league_home_goals_conceded_avg,league_away_goals_conceded_avg,team_attack_strength,team_defense_strength,opponent_attack_strength,opponent_defense_strength,team_lambda,opponent_lambda
0,"Serie C, Girone B",2022-09-04,alessandria,imolese,0.0,1401.513138,1385.984449,15.528689,0,0.842105,...,0.444444,1.111111,0.802339,1.79152,1.246356,0.998858,0.675698,1.311953,1.311953,1.209143
1,"Serie C, Girone B",2022-09-04,aquila_montevarchi,gubbio,0.0,1432.585346,1473.678249,-41.092903,0,1.105263,...,0.444444,1.111111,0.802339,1.79152,0.655977,1.028236,0.998858,1.115160,0.586926,1.840001
2,"Serie C, Girone B",2022-09-04,carrarese,cesena,1.0,1415.944287,1507.312415,-91.368128,0,1.000000,...,0.444444,1.111111,0.802339,1.79152,0.655977,1.116370,1.028236,1.049563,0.552401,2.056471
3,"Serie C, Girone B",2022-09-04,cesena,carrarese,0.0,1507.312415,1415.944287,91.368128,1,1.473684,...,0.444444,1.111111,0.802339,1.79152,1.028236,1.049563,0.655977,1.116370,2.056471,0.552401
4,"Serie C, Girone B",2022-09-04,fermana,fiorenzuola,0.0,1376.394658,1426.747051,-50.352394,0,0.736842,...,0.444444,1.111111,0.802339,1.79152,0.721574,0.851967,0.499429,1.771137,1.025395,0.762286


In [24]:
data.shape

(290, 35)

In [25]:
def build_dataset(df):
    
    df = df.drop(['league', 'date', 'team', 'opponent'], axis=1)
    X = df.drop(['result'], axis=1).to_numpy()
    Y = np.array(df['result']) / 0.5
    
    X = torch.tensor(X).float()
    Y = torch.tensor(Y).long()
    
    
    return X, Y

In [26]:
X, Y = build_dataset(data)

In [27]:
PATH = "../src/model/trained_models/3_linear_layer.pt"
# model.load_state_dict(torch.load(PATH))
model = torch.load(PATH)
model.train()

Sequential(
  (0): Linear(in_features=30, out_features=100, bias=False)
  (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Tanh()
  (3): Linear(in_features=100, out_features=100, bias=False)
  (4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): Tanh()
  (6): Linear(in_features=100, out_features=100, bias=False)
  (7): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Tanh()
  (9): Linear(in_features=100, out_features=3, bias=True)
)

In [89]:
@torch.no_grad()
def make_predictions(x):
    logits = model(x)
    preds = []
#     preds = torch.argmax(logits, dim=1)
    preds = torch.softmax(logits, dim=1)
    
    return preds

In [90]:
predictions = make_predictions(X)
data[['0', '1', '2']] = predictions.numpy()

In [91]:
data.to_csv("../data/past_predictions.csv")
predictions_data = data[['league', 'date', 'team', 'opponent', 'result', 'home', '0', '1', '2']]
predictions_data.to_csv("../data/past_predictions.csv")

## Joining to Future

In [54]:
future = pd.read_csv("../data/future_predictions.csv", index_col=0)
future.head()

,league,date,team,opponent,home,0,1,2
0,"Serie C, Girone B",2023-01-08,siena,reggiana,0,0.136038,0.517463,0.346498
1,"Serie C, Girone B",2023-01-08,san_donato_tavarnelle,sassari_torres,1,0.184710,0.342512,0.472778
2,"Serie C, Girone B",2023-01-08,gubbio,fermana,1,0.808809,0.105148,0.086042
3,"Serie C, Girone B",2023-01-08,imolese,lucchese,1,0.126196,0.307486,0.566317
4,"Serie C, Girone B",2023-01-08,olbia,aquila_montevarchi,1,0.665615,0.134607,0.199779


In [61]:
elos = pd.read_csv("../data/elos_matches.csv", index_col=0, parse_dates=['date'], dayfirst=False)
elos['date'] = elos['date'].dt.date
elos.head()

,league,date,team,opponent,result,elo_team,elo_opponent,elo_diff,home
0,Serie A,1997-08-31,atalanta,bologna,1.0,1500.0,1500.0,0.0,1
5,Serie A,1997-08-31,l_r_vicenza,sampdoria,0.0,1500.0,1500.0,-0.0,0
6,Serie A,1997-08-31,milan,piacenza,0.5,1500.0,1500.0,-0.0,0
7,Serie A,1997-08-31,roma,empoli,1.0,1500.0,1500.0,-0.0,0
8,Serie A,1997-08-31,fiorentina,udinese,1.0,1500.0,1500.0,-0.0,0


In [78]:
start_date = pd.to_datetime(f"2022-07-30").date()

predictions_data = data[['league', 'date', 'team', 'opponent', 'result', 'home', 'loss', 'draw', 'win']]
elos_c = elos[elos['league']=='Serie C, Girone B']
elos_c = elos_c[elos_c['date']>=start_date].reset_index(drop=True)
combined = pd.concat([elos_c.set_index(['league', 'date', 'team', 'opponent', 'result']),
                      predictions_data.set_index(['league', 'date', 'team', 'opponent', 'result'])],
                     axis=1,
#                      columns = ['league', 'date', 'team', 'opponent', 'result']
                    ).reset_index()
combined.head()

,league,date,team,opponent,result,elo_team,elo_opponent,elo_diff,home,home,loss,draw,win
0,"Serie C, Girone B",2022-09-04,rimini,san_donato_tavarnelle,0.5,1527.078858,1549.720500,-22.641641,0,0,0.782737,0.144604,0.072659
1,"Serie C, Girone B",2022-09-04,gubbio,aquila_montevarchi,1.0,1473.678249,1432.585346,41.092903,1,1,0.075898,0.145895,0.778206
2,"Serie C, Girone B",2022-09-04,sassari_torres,virtus_entella,0.0,1549.128577,1493.601266,55.527311,0,0,0.627835,0.197015,0.175150
3,"Serie C, Girone B",2022-09-04,fermana,fiorenzuola,0.0,1376.394658,1426.747051,-50.352394,0,0,0.724157,0.181032,0.094811
4,"Serie C, Girone B",2022-09-04,lucchese,reggiana,0.0,1444.953343,1535.810856,-90.857513,0,0,0.877273,0.090440,0.032287


In [79]:
combined.tail()

,league,date,team,opponent,result,elo_team,elo_opponent,elo_diff,home,home,loss,draw,win
285,"Serie C, Girone B",2022-12-17,fermana,sassari_torres,1.0,1365.520797,1476.896983,-111.376186,1,1,0.368500,0.365424,0.266076
286,"Serie C, Girone B",2022-12-23,sassari_torres,virtus_entella,0.0,1457.246697,1535.560893,-78.314195,1,1,0.806222,0.147915,0.045863
287,"Serie C, Girone B",2022-12-23,virtus_entella,sassari_torres,1.0,1535.560893,1457.246697,78.314195,0,0,0.044062,0.111864,0.844074
288,"Serie C, Girone B",2022-12-23,san_donato_tavarnelle,rimini,1.0,1427.191531,1485.185717,-57.994186,0,0,0.173215,0.215140,0.611644
289,"Serie C, Girone B",2022-12-23,rimini,san_donato_tavarnelle,0.0,1485.185717,1427.191531,57.994186,1,1,0.684631,0.186265,0.129105
